### The purpose of this notebook is to perform credit risk analysis to predict customer probability of default using the LendingClub data

The objective of quantitative credit scoring models is to assign credit applicants to one of two groups: a “good credit” group that is likely to repay the financial obligation, or a “bad credit” group that should be denied credit because of a high likelihood of defaulting on the financial obligation.
Even a fraction of a percent increase in credit scoring accuracy is a significant accomplishment.
The advantages of credit scoring include reducing the cost of credit analysis, enabling faster credit decisions, closer monitoring of existing accounts, and prioritizing collections
To pursue even small improvements in credit scoring accuracy, the practitioner must explore other neural network architectures beyond the conventional MLP, as well as nonparametric statistical models and classification trees.

In [1]:
#Import the relevant library for data exploration and modelling
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix, precision_recall_curve, auc
from sklearn.feature_selection import f_classif
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from scipy.stats import chi2_contingency

In [5]:
#Import the data and load it into a dataframe
loan_data = pd.read_csv('loans_full_schema.csv')

In [11]:
pd.options.display.max_columns = None

In [12]:
loan_data.shape

(10000, 55)

The data contains 10,000 entities(rows) and 55 attributes(columns)

In [13]:
#Return the first 5 entities of the data
loan_data.head()

emp_title  emp_length state homeownership  annual_income  \
0  global config engineer          3.0    NJ      MORTGAGE        90000.0   
1   warehouse office clerk        10.0    HI          RENT        40000.0   
2                 assembly         3.0    WI          RENT        40000.0   
3         customer service         1.0    PA          RENT        30000.0   
4     security supervisor         10.0    CA          RENT        35000.0   

   verified_income  debt_to_income  annual_income_joint  \
0         Verified           18.01                  NaN   
1     Not Verified            5.04                  NaN   
2  Source Verified           21.15                  NaN   
3     Not Verified           10.16                  NaN   
4         Verified           57.96              57000.0   

  verification_income_joint  debt_to_income_joint  delinq_2y  \
0                       NaN                   NaN          0   
1                       NaN                   NaN          0   
2                       NaN                   NaN          0   
3                       NaN                   NaN          0   
4                  Verified                 37.66          0   

   months_since_last_delinq  earliest_credit_line  inquiries_last_12m  \
0                      38.0                  2001                   6   
1                       NaN                  1996                   1   
2                      28.0                  2006                   4   
3                       NaN                  2007                   0   
4                       NaN                  2008                   7   

   total_credit_lines  open_credit_lines  total_credit_limit  \
0                  28                 10               70795   
1                  30                 14               28800   
2                  31                 10               24193   
3                   4                  4               25400   
4                  22                 16               69839   

   total_credit_utilized  num_collections_last_12m  \
0                  38767                         0   
1                   4321                         0   
2                  16000                         0   
3                   4997                         0   
4                  52722                         0   

   num_historical_failed_to_pay  months_since_90d_late  \
0                             0                   38.0   
1                             1                    NaN   
2                             0                   28.0   
3                             1                    NaN   
4                             0                    NaN   

   current_accounts_delinq  total_collection_amount_ever  \
0                        0                          1250   
1                        0                             0   
2                        0                           432   
3                        0                             0   
4                        0                             0   

   current_installment_accounts  accounts_opened_24m  \
0                             2                    5   
1                             0                   11   
2                             1                   13   
3                             1                    1   
4                             1                    6   

   months_since_last_credit_inquiry  num_satisfactory_accounts  \
0                               5.0                         10   
1                               8.0                         14   
2                               7.0                         10   
3                              15.0                          4   
4                               4.0                         16   

   num_accounts_120d_past_due  num_accounts_30d_past_due  \
0                         0.0                          0   
1                         0.0                          0   
2                         0.0            

In [14]:
loan_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 55 columns):
emp_title                           9167 non-null object
emp_length                          9183 non-null float64
state                               10000 non-null object
homeownership                       10000 non-null object
annual_income                       10000 non-null float64
verified_income                     10000 non-null object
debt_to_income                      9976 non-null float64
annual_income_joint                 1495 non-null float64
verification_income_joint           1455 non-null object
debt_to_income_joint                1495 non-null float64
delinq_2y                           10000 non-null int64
months_since_last_delinq            4342 non-null float64
earliest_credit_line                10000 non-null int64
inquiries_last_12m                  10000 non-null int64
total_credit_lines                  10000 non-null int64
open_credit_lines          

In [15]:
#Return a list of columns that have more than 80% null values
null_values = loan_data.isnull().mean()
null_values[null_values>0.8]

annual_income_joint          0.8505
verification_income_joint    0.8545
debt_to_income_joint         0.8505
dtype: float64

The above results portraty that data pertaining to joint application contain a significant number of null values.

In [19]:
#The columns which contain more than 80% null values
loan_data.dropna(thresh = loan_data.shape[0]*0.2, how = 'all', axis = 1, inplace = True)

'''
Getting rid of columns which might not necessarily have an impact on the customer's credit stability and redundant like sub_grade, recoveries
'''
loan_data.drop(columns = ['sub_grade', 'emp_title'], inplace = True)

In [20]:
#Reexplore the data
loan_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 50 columns):
emp_length                          9183 non-null float64
state                               10000 non-null object
homeownership                       10000 non-null object
annual_income                       10000 non-null float64
verified_income                     10000 non-null object
debt_to_income                      9976 non-null float64
delinq_2y                           10000 non-null int64
months_since_last_delinq            4342 non-null float64
earliest_credit_line                10000 non-null int64
inquiries_last_12m                  10000 non-null int64
total_credit_lines                  10000 non-null int64
open_credit_lines                   10000 non-null int64
total_credit_limit                  10000 non-null int64
total_credit_utilized               10000 non-null int64
num_collections_last_12m            10000 non-null int64
num_historical_failed_to_pay 

Five columns have been dropped

#### Identify the target variable

In [21]:
# explore the unique values in loan_status column
loan_data['loan_status'].value_counts(normalize = True)

Current               0.9375
Fully Paid            0.0447
In Grace Period       0.0067
Late (31-120 days)    0.0066
Late (16-30 days)     0.0038
Charged Off           0.0007
Name: loan_status, dtype: float64

The above output outline the 